In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

BATCH_SIZE = 512 #每批处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu ")#是否用GPU
EPOCHS = 10 #训练数据葉的轮次

pipeline = transforms.Compose([
transforms . ToTensor (), #将图片转换成tensor
transforms .Normalize(( 0.1307 ,), ( 0.3081,)) # 正则化：降低模型复杂度
])

from torch.utils.data import DataLoader

#下载数据集
train_set = datasets.MNIST("./data/手写字体识别", train=True, download=True, transform=pipeline)
test_set = datasets.MNIST("./data/手写字体识别", train=False, download=True, transform=pipeline)
#加载数据
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)  #1：灰度图片的通道,10：输出通道,5： kerne
        self. conv2 = nn.Conv2d( 10 , 20 , 3 ) #10：输入通道,20：输出通道,3 ： Kernel
        self. fc1 = nn.Linear(20*10*10, 500 ) #20*10*10：输入通道,500：输出通道
        self. fc2 = nn.Linear (500, 10 ) # 500：输入通道,10：输出通蓿

    def forward(self, x):
        input_size = x.size(0)  #batch_size
        x = self.conv1(x)  # 軽入:bvtch*1*28*28，输出:batch*10*24*24 ( 28 - 5 +
        x = F.relu(x)  # 保持shape不变,输出:bvtch*10*24*24
        x = F.max_pool2d(x, 2, 2)  #输入:bvtch*10*24*24 输出:bvtch*10*12*12
        x = self.conv2(x)  # 输入:bvtch*10*12*12 输出: bvtch*20*10*10 (12-3+ x = F.relu(x) x = x.view(input_size, -1) # 拉平,-1 自动计算维度,20*10*10= 2000 x = self. fc1 (x) # 输入:bvtch*2000 输出:bvtch*500
        x = F.relu(x)  # 保持shpve不变
        x=x.view(input_size,-1)
        x=self.fc1(x)
        x=F.relu(x)
        x=self.fc2(x)
        # 输入:bvtch*500 输出:bvtch*10
        output = F.log_softmax(x, dim=1)  #计算分类后,每个数字的概率值
        return output


In [6]:
# 6定义优化器
model = Digit().to(DEVICE)
optimizer = optim.Adam(model.parameters())

# 7定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    #模型训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):
        #部署到DEVICE上去
        data, target = data.to(device), target.to(device)
        #梯度初始化为0
        optimizer.zero_grad()
        #训练后的结果
        output = model(data)
        #计算损失
        loss = F.cross_entropy(output, target)
        #反向传播
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print("Train Epoch : {} \t Loss : {:.6f}".format(epoch, loss.item()))

# 8定义测试方法
def test_model(model, device, test_loader):
    #模型验证	—
    model.eval()
    #正确率
    correct = 0.0
    #测试损失
    test_loss = 0.0
    with torch.no_grad(): #不会计算梯度,也不会进行反向传播
        for data, target in test_loader:
            #部署到device上
            data, target = data.to(device), target.to(device)
            #测试数据
            output = model(data)
            #计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            #找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1] # 值, 索弓I
            #pred = torch.max(ouput, dim=1)
            #pred = output・vrgmvx(dim=1)
            #累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test —— Average loss : {:.4f}, Accuracy : {:.3f}\n".format(test_loss, 100.0 * correct / len(test_loader.dataset)))


for epoch in range(1, EPOCHS + 1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model, DEVICE, test_loader)

Train Epoch : 1 	 Loss : 2.318762
Test —— Average loss : 0.0002, Accuracy : 97.150

Train Epoch : 2 	 Loss : 0.109728
Test —— Average loss : 0.0001, Accuracy : 98.050

Train Epoch : 3 	 Loss : 0.051256
Test —— Average loss : 0.0001, Accuracy : 98.450

Train Epoch : 4 	 Loss : 0.051196
Test —— Average loss : 0.0001, Accuracy : 98.800

Train Epoch : 5 	 Loss : 0.032738
Test —— Average loss : 0.0001, Accuracy : 98.680

Train Epoch : 6 	 Loss : 0.026315


KeyboardInterrupt: 

In [2]:
import torch
import torch.nn.functional as F
b = torch.tensor([1.1], requires_grad=True)
b = b * 2

# with torch.no_grad():
# b.mul_(2)

b
a = torch.tensor([[1,5,62,54], [2,6,2,6], [2,65,2,6]])
print(a)
torch.max(a, 1)

x= torch.Tensor( [ [1,2,3,4],[1,2,3,4],[1,2,3,4]])

y1= torch.softmax(x, dim = 1) #对每一列进行softmax
print(y1)

number=3.24455
print("Test —— Average loss : {:.4f}, Accuracy\n".format(number))



tensor([[ 1,  5, 62, 54],
        [ 2,  6,  2,  6],
        [ 2, 65,  2,  6]])
tensor([[0.0321, 0.0871, 0.2369, 0.6439],
        [0.0321, 0.0871, 0.2369, 0.6439],
        [0.0321, 0.0871, 0.2369, 0.6439]])
Test —— Average loss : 3.2445, Accuracy



In [3]:
import os

Path1 = '/home/sys'
Path2 = 'develop'
Path3 = 'code'

Path10 = Path1 + Path2 + Path3
Path20 = os.path.join(Path1,Path2,Path3)
print ('Path10 = ',Path10)
print ('Path20 = ',Path20)



# 案例一：获取字典中key值是小写字母的键值对
dict1 = {"a":10,"B":20,"C":True,"D":"hello world","e":"python教程"}
dict2 = {key:value for key,value in dict1.items() if key.islower()}
dict3 = {x:"value"+x for x in ["train","test"]}
print(dict2)
print(dict3)

lst = [1, 2, 3]
for i in iter(lst):
    print(i)

it=next(iter([1,2,3,4,5]))
print(it)
# while True:
#     try:
#         x=next(it)
#         print(x)
#     except StopIteration:
#         break

Path10 =  /home/sysdevelopcode
Path20 =  /home/sys\develop\code
{'a': 10, 'e': 'python教程'}
{'train': 'valuetrain', 'test': 'valuetest'}
1
2
3
1


In [16]:
#flatten函数的操作
import torch.nn as nn
t = torch.tensor([[[1, 2, 2, 1],
                   [3, 4, 4, 3],
                   [1, 2, 3, 4]],
                  [[5, 6, 6, 5],
                   [7, 8, 8, 7],
                   [5, 6, 7, 8]]])

nn.fc=nn.Sequential(
    nn.Flatten(start_dim=1),
    nn.Softmax()
)
t=nn.fc(t)
print(t, t.shape)




C:\anaconda3\lib\site-packages\torch\nn\modules\container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


RuntimeError: "softmax_lastdim_kernel_impl" not implemented for 'Long'

In [24]:
from torch import autograd

x=torch.randn([3,4])
print(x)
bn=nn.BatchNorm1d(4)
output=bn(x)
print(output)

tensor([[-0.0518,  1.5241,  0.6666, -0.0820],
        [-1.0687,  0.6574,  1.6889, -0.4477],
        [ 0.1083, -0.4594,  0.1849, -0.1963]])
tensor([[ 0.5480,  1.1701, -0.2873,  1.0471],
        [-1.4030,  0.1027,  1.3428, -1.3464],
        [ 0.8551, -1.2729, -1.0555,  0.2993]],
       grad_fn=<NativeBatchNormBackward>)
